<a href="https://colab.research.google.com/github/greatsong/meet_deep3/blob/main/210206_%EB%B0%91%EB%94%A53_%ED%95%84%EC%82%AC_4%EC%9D%BC%EC%B0%A8_%EC%86%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# step 1

In [ ]:
class Variable : 
  def __init__(self, data) :
    self.data = data

In [ ]:
import numpy as np
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [ ]:
x.data  = np.array(2)
print(x.data)

2


In [ ]:
import numpy as np
x = np.array(1)
x.ndim

0

In [ ]:
x = np.array([1,2,3])
x.ndim

1

In [ ]:
x = np.array([[1,2,3],
             [4,5,6]])
x.ndim

2

In [ ]:
# step 2

In [ ]:
class Function :
  def __call__(self, input) : 
    x = input.data
    y = x ** 2
    output = Variable(y)
    return output

In [ ]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [ ]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [ ]:
class Function :
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output
  def forward(self, x) :
    raise NotImplementedError()

In [ ]:
class Square(Function) :
  def forward(self, x ) :
    return x ** 2

In [ ]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [ ]:
# step 3

In [ ]:
class Exp(Function) :
  def forward(self, x) :
    return np.exp(x)

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


In [ ]:
# step 4

In [ ]:
def numerical_diff(f, x, eps = 1e-4) :
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

In [ ]:
f = Square()
x = Variable(np.array(2))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [ ]:
def f(x) :
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [ ]:
# step 5

In [ ]:
# step 6

In [ ]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None

In [ ]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [ ]:
class Square(Function) :
  def forward(self, x) :
    y = x ** 2
    return y

  def backward(self, gy) :
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [ ]:
class Exp(Function) :
  def forward(self, x) :
    y = np.exp(x)
    return y

  def backward(self, gy) : 
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [ ]:
y.grad = np.array(1)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [ ]:
# step 7

In [ ]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

In [ ]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [ ]:
# 그냥 실행하면 오류가 발생 / 왜냐하면 스퀘어, 지수 함수가 업데이트 안되어서!
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [ ]:
assert y.creator == C

In [ ]:
print(a.creator)

In [ ]:
y.grad = np.array(1)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [ ]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [ ]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [ ]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    f = self.creator
    if f is not None : 
      x = f.input
      x.grad = f.backward(self.grad)
      # 중간 과정 확인하기 위한 print 추가
      print(f, x.grad)
      x.backward()

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1)
y.backward()
print(x.grad)

<__main__.Square object at 0x7f51c8a07390> 2.568050833375483
<__main__.Exp object at 0x7f51c8a071d0> 3.297442541400256
<__main__.Square object at 0x7f51c8a07240> 3.297442541400256
3.297442541400256


In [ ]:
# step 8

In [ ]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      print(f, funcs)
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None : 
        funcs.append(x.creator)
      # 중간 과정 확인하기 위한 print 추가

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1)
y.backward()
print(x.grad)

<__main__.Square object at 0x7f51c89d2f28> []
<__main__.Exp object at 0x7f51c89d29b0> []
<__main__.Square object at 0x7f51c89d2198> []
3.297442541400256


In [ ]:
# step 9

In [ ]:
import numpy as np

In [ ]:
def square(x) :
  f = Square()
  return f(x)

def exp(x) :
  f = Exp()
  return f(x)

In [12]:
def square(x) :
  return Square()(x)

def exp(x) :
  return Exp()(x)


In [ ]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [ ]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      print(f, funcs)
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None : 
        funcs.append(x.creator)
      # 중간 과정 확인하기 위한 print 추가

In [ ]:
class Square(Function) :
  def forward(self, x) :
    y = x ** 2
    return y

  def backward(self, gy) :
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [ ]:
class Exp(Function) :
  def forward(self, x) :
    y = np.exp(x)
    return y

  def backward(self, gy) : 
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [ ]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1)
y.backward()
print(x.grad)

<__main__.Square object at 0x7fa582af7128> []
<__main__.Exp object at 0x7fa582af7080> []
<__main__.Square object at 0x7fa582af7b70> []
3.297442541400256


In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1)
y.backward()
print(x.grad)

<__main__.Square object at 0x7fa582b83d68> []
<__main__.Exp object at 0x7fa582af7208> []
<__main__.Square object at 0x7fa582af7470> []
3.297442541400256


In [ ]:
class Variable :
  def __init__(self, data) : 
    if data is not None : 
      if not isinstance(data, np.ndarray) :
        #raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        raise TypeError(f'{data} {type(data)}은 지원하지 않습니다.')
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    if self.grad is None :
      self.grad = np.ones_like(self.data)
      print('np.ones_like print', self.grad)

    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      print(f, funcs)
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None : 
        funcs.append(x.creator)
      # 중간 과정 확인하기 위한 print 추가

In [ ]:
x = Variable(np.array(1))
x = Variable(None)
x = Variable(1)

TypeError: ignored

In [ ]:
x = Variable(1.0)

TypeError: ignored

In [ ]:
# 0.5는 ndarray가 아니므로 오류 발생. 따라서 리스트에 값을 넣어주어야 함
x = Variable(np.array([0.5]))

In [ ]:
y = square(exp(square(x)))
y.backward()
print(x.grad)

np.ones_like print [1.]
<__main__.Square object at 0x7fa57ad227f0> []
<__main__.Exp object at 0x7fa582af7198> []
<__main__.Square object at 0x7fa582af70b8> []
[3.29744254]


In [ ]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [ ]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [ ]:
def as_array(x) :
  if np.isscalar(x) :
    return np.array(x)
  return x

In [ ]:
np.isscalar(1)

True

In [ ]:
np.isscalar(np.float64(1.0))

True

In [ ]:
np.isscalar(np.array(1))

False

In [ ]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(as_array(y))
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [ ]:
# step 10 Unit Test!

In [ ]:
import unittest

class SquareTest(unittest.TestCase) :
  def test_forward(self) :
    x = Variable(np.array([2.0]))
    y = square(x)
    expected = np.array([4.0])
    self.assertEqual(y.data, expected)

  def test_backward(self) :
    x = Variable(np.array([3.0]))
    y = square(x)
    y.backward()
    expected = np.array([6.0])
    self.assertEqual(x.grad, expected)

# 테스트하기 위한 코드
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..

np.ones_like print [1.]
<__main__.Square object at 0x7fa57ad17240> []



----------------------------------------------------------------------
Ran 2 tests in 0.011s

OK


In [ ]:
def numerical_diff(f, x, eps = 1e-4) :
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

In [ ]:
import unittest

class SquareTest(unittest.TestCase) :
  def test_forward(self) :
    x = Variable(np.array([2.0]))
    y = square(x)
    expected = np.array([4.0])
    self.assertEqual(y.data, expected)

  def test_backward(self) :
    x = Variable(np.array([3.0]))
    y = square(x)
    y.backward()
    expected = np.array([6.0])
    self.assertEqual(x.grad, expected)

  def test_gradient_check(self) :
    x = Variable(np.random.rand(1))
    y = square(x)
    y.backward()
    num_grad= numerical_diff(square, x)
    flg = np.allclose(x.grad, num_grad)
    self.assertTrue(flg)

# 테스트하기 위한 코드
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


...

np.ones_like print [1.]
<__main__.Square object at 0x7fa57acab080> []
np.ones_like print [1.]
<__main__.Square object at 0x7fa57acabe80> []



----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


In [ ]:
# step 11

In [ ]:
class Variable :
  def __init__(self, data) : 
    if data is not None : 
      if not isinstance(data, np.ndarray) :
        #raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        raise TypeError(f'{data} {type(data)}은 지원하지 않습니다.')
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    if self.grad is None :
      self.grad = np.ones_like(self.data)
      print('np.ones_like print', self.grad)

    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      print(f, funcs)
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None : 
        funcs.append(x.creator)
      # 중간 과정 확인하기 위한 print 추가

In [18]:
def as_array(x) :
  if np.isscalar(x) :
    return np.array(x)
  return x

def add(x0, x1) :
  return Add()(x0, x1)

def square(x) :
  return Square()(x)

def exp(x) :
  return Exp()(x)


In [ ]:

# 다중 입력, 다중 출력을 위해 클래스 수정
class Function : 
  def __call__(self, inputs) : 
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]
    for output in outputs : 
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return output

  def forward(self, xs) :
    raise NotImplementedError()

  def backward(self, gys) :
    raise NotImplementedError()

In [ ]:
class Add(Function) :
  def forward(self, xs) :
    x0, x1 = xs
    y = x0 + x1
    return y,

In [ ]:
import numpy as np
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # 신기하게 forward 함수를 호출 안했는데 실행이 되네??;;
y = ys
print(y.data)

5


In [ ]:
# step 12 

In [ ]:
# 다중 입력, 다중 출력을 위해 클래스 수정
class Function : 
  def __call__(self, *inputs) : 
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]
    for output in outputs : 
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs) :
    raise NotImplementedError()

  def backward(self, gys) :
    raise NotImplementedError()

In [ ]:
def f(*x) :
  print(x)

In [ ]:
f(1,2,3)

(1, 2, 3)


In [ ]:
import numpy as np


In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [5]:
# 다중 입력, 다중 출력을 위해 클래스 수정
class Function : 
  def __call__(self, *inputs) : 
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple) :
      ys = (ys, )
    outputs = [Variable(as_array(y)) for y in ys]
    for output in outputs : 
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs) :
    raise NotImplementedError()

  def backward(self, gys) :
    raise NotImplementedError()

In [ ]:
class Add(Function) :
  def forward(self, x0, x1) :
    y = x0 + x1
    return y

In [11]:
def add(x0, x1) :
  return Add()(x0, x1)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

y = add(x0, x1)

print(y.data)

5


In [1]:
# 4일차 : step 13

In [6]:
class Add(Function) :
  def forward(self, x0, x1) :
    y = x0 + x1
    return y
  def backward(self, gy) :
    return gy, gy

In [4]:
class Variable :
  def __init__(self, data) : 
    if data is not None : 
      if not isinstance(data, np.ndarray) :
        #raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        raise TypeError(f'{data} {type(data)}은 지원하지 않습니다.')
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    if self.grad is None :
      self.grad = np.ones_like(self.data)
      print('np.ones_like print', self.grad)

    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs) : 
        x.grad = gx
        if x.creator is not None : 
          funcs.append(x.creator)

In [8]:
class Square(Function) :
  def forward(self, x) :
    y = x ** 2
    return y

  def backward(self, gy) :
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx

In [ ]:
def as_array(x) :
  if np.isscalar(x) :
    return np.array(x)
  return x

def add(x0, x1) :
  return Add()(x0, x1)

def square(x) :
  return Square()(x)

def exp(x) :
  return Exp()(x)


In [16]:
import numpy as np
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

np.ones_like print 1.0
13.0
4.0
6.0


In [17]:
# step 14

In [20]:
class Variable :
  def __init__(self, data) : 
    if data is not None : 
      if not isinstance(data, np.ndarray) :
        #raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        raise TypeError(f'{data} {type(data)}은 지원하지 않습니다.')
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    if self.grad is None :
      self.grad = np.ones_like(self.data)
      print('np.ones_like print', self.grad)

    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs) : 
        if x.grad is None :
          x.grad = gx
        else : 
          x.grad = x.grad + gx
        if x.creator is not None : 
          funcs.append(x.creator)

In [21]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

np.ones_like print 1.0
2.0


In [22]:
x = Variable(np.array(3.0))
y = add(add(x,x),x)
y.backward()
print(x.grad)

np.ones_like print 1.0
3.0


In [23]:
class Variable :
  def __init__(self, data) : 
    if data is not None : 
      if not isinstance(data, np.ndarray) :
        #raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        raise TypeError(f'{data} {type(data)}은 지원하지 않습니다.')
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    if self.grad is None :
      self.grad = np.ones_like(self.data)
      print('np.ones_like print', self.grad)

    funcs = [self.creator]
    while funcs :
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs) : 
        if x.grad is None :
          x.grad = gx
        else : 
          x.grad = x.grad + gx
        if x.creator is not None : 
          funcs.append(x.creator)
  def cleargrad(self) :
    self.grad = None

In [26]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x,x),x)
y.backward()
print(x.grad)

np.ones_like print 1.0
2.0
np.ones_like print 1.0
3.0
